# Topical lectures on modern C++ with a bias towards contributing to HEP codes 

Presenter: David Lange (Princeton University) <br>
Contact me: david.lange@cern.ch

May 2023 HSF-India tutorials

# Goals for these lectures
  * Introduce some of the concepts needed for CUDA lectures later this week
  * Introduce how the evolving C++ standard helps to write clear and performant code
  * Introduce some needed techniques for developing codes in a larger ecosystem 
  
These will be "topical" rather than exhaustive. Understanding some of these concepts in detail takes time and practice
  
# Topics for these lectures
  * [Language introduction](#lang_intro)
  * [Declaration syntax](#variables)
  * [Loops](#loops)
  * [Standard Template Library](#stl)
  * [Memory considerations](#memory)
  * [Modern pointers](#pointers)
  * Compile and link -- in part 2
  * Optimization -- in part 3
  
  [Another Cell](#another_cell)

<a id='lang_intro'></a>
# C++ language and its evolution

  * The C++ programming language was devised by Bjarne Stroustrup - then an employee of Bell Labs (AT&T). Stroustrup started working on C with Classes in 1979 and the first commercial release of the C++ language was in October 1985
    * Strengths of C++ include how it allows researchers to write efficient code without losing high-level abstraction; its support for developing both large scale (eg, HPC) applications and low-level drivers and embedded systems; and its support for high-performance/latency-critical applications.
    * Now a mature ecosystem: Many available utilities to help programmers write and debug. These include debuggers, memory checkers, coverage, static analysis, profiling tools, etc
    * The C++ language continues to evolve to support modern architecturs and improved code quality/performance
  * HEP moved to C++ (from Fortran) as its primary programming language for computationally intensive applications starting in the late 1990s. Now the field has 100s of millions of lines of code in production. It would be a significant undertaking to rewrite this code base.

# C++ language and its evolution (II)
“The Evolution of C++Past, Present, and Future”, B. Stroustrup, CppCon16

<center><img src="./images/cppmap.png" width="500" /></center>

# C++ efficiency

<center><img src="./images/co2.png" width="500" /></center>

# Example of C++ in HEP

<center><img src="./images/slocs.png" width="500" /></center>

# Resources

## These are some web-based (and open) resources that you might find useful for following up on these lectures
  * C++ standard pages https://github.com/isocpp/CppCoreGuidelines/blob/master/CppCoreGuidelines.md
  * HSF C++ course: https://github.com/hsf-training/cpluspluscourse (including videos)
  * github course I found
  * https://en.cppreference.com/w/ : Usage and syntax including how it evolves with C++ standards and including (non-trivial) examples

# C++ in notebooks

  * Much of this course will be using C++ in notebooks. This is great for learning and prototyping.
  * It does let us take shortcuts - for example, often proper header file includes can be skipped, compiling and linking happens behind the scenes so we do not need to worry about. 
    * Do not worry - we will come back to these topics at the end (hopefully)

<a id='variables'></a>
# Declaring variables

  * C++ enforces type safety. This includes compile time checking

In [1]:
int16_t a = 1; // Prefer fixed-width (signed or unsigned) integers instead of native types
int16_t a2(1); // Direct initialization
int16_t a3=int(1); // Copy initialization
int16_t a4{1}; // Copy-list initialization

std::cout << a << " " << a2 << " " << a3 << " " << a4 << std::endl;

1 1 1 1


In [2]:
//Pitfalls that the compiler can help you avoid
uint16_t c = -1;

std::cout << c << std::endl;

65535


In [3]:
// Prefer {} to = as it allows compile time checks
uint16_t c{-1}; // {} allows the compiler to find your mistake at compile time

input_line_53:3:12: error: constant expression evaluates to -1 which cannot be narrowed to type 'uint16_t' (aka 'unsigned short') [-Wc++11-narrowing]
uint16_t c{-1}; // {} allows the compiler to find your mistake at compile time
           ^~
input_line_53:3:12: note: insert an explicit cast to silence this issue
uint16_t c{-1}; // {} allows the compiler to find your mistake at compile time
           ^~
           static_cast<uint16_t>( )


In [4]:
//Same for floats
float f1=10e40;
std::cout << f1 << std::endl;

inf


In [5]:
float f2{10e40};

input_line_55:2:11: error: constant expression evaluates to 1.000000e+41 which cannot be narrowed to type 'float' [-Wc++11-narrowing]
 float f2{10e40};
          ^~~~~
input_line_55:2:11: note: insert an explicit cast to silence this issue
 float f2{10e40};
          ^~~~~
          static_cast<float>( )


# Declaring arrays has also gotten easier

In [6]:
//helper function
template<class T>
void print(T *arr, int16_t len) {
   for ( int16_t i=0; i<len; i++ ) std::cout << arr[i] << " ";
   std::cout << std::endl; 
}

In [7]:
//An old way - lots of typing, and hard to read
double a[4];
a[0]=1.0;
a[1]=1.0;
a[2]=1.0;
a[3]=1.0;
print(a,4);


1 1 1 1 


In [8]:
//Another old way
double a[4];
for ( int i=0; i<4; i++ ) { a[i] = 1.0; }
print(a,4);

1 1 1 1 


In [9]:
//Example newer approaches
double a[4]{1.0, 1.0, 1.0, 1.0};
print(a,4);

1 1 1 1 


In [10]:
//or even shorter
double a[]{1.0,1.0,1.0,1.0};
std::cout << a[0] << " " << a[1] << " " << a[2] << " " << a[3] << std::endl;

1 1 1 1


In [11]:
//it is also easy to use the default initializer
double a[4]{0};
print(a,4);
double b[4]{};
print(b,4);

0 0 0 0 
0 0 0 0 


In [12]:
//maybe not the behavior that you expected?
double a[4]{1.0};
print(a,4);

1 0 0 0 


  * We will come back to arrays later..

<a id='loops'></a>
# Lets talk about loops

In [13]:
//Basic old-style loop
double a[4]{1.0,2.0,4.0,9.0};

double sum_sq{0.0};
for (int i=0; i<4; i++) 
    sum_sq += a[i]*a[i];
print(&sum_sq,1);


102 


In [14]:

//Range-based loops can simplify this syntax
double sum_sq{0.0};
for ( double v: a) 
    sum_sq += v*v;
print(&sum_sq,1);

102 


In [15]:

//You can also inline the values to loop over
double sum_sq{0.0};
for ( double v: {1.0,2.0,4.0,9.0}) 
    sum_sq += v*v;
print(&sum_sq,1);

102 


In [16]:

//And let the compiler understand the type using "auto"
double sum_sq{0.0};
for ( auto v: {1.0,2.0,4.0,9.0}) 
    sum_sq += v*v;
print(&sum_sq,1);

102 


# Pitfalls in loops

In [17]:
float x = 0.0f;
for (int i = 0; i < 40000000; i++) 
    x += 1.0f;
std::cout << (int)x << std::endl;



16777216


In [18]:
//Think about when to use floating point vs fixed point variables
int64_t x = 0.0f;
for (int i = 0; i < 40000000; i++) 
    x += 1;
std::cout << (int)x << std::endl;



40000000


  * This pitfall isn't really about loops, it is about floating point representations
  * Floating point precision is finite! For example, floats in C++ have 1 sign bit, 8-bit exponent, 23-bit significand

In [19]:
std::cout << 16777217 << std::endl;
std::cout << (int) 16777217.0f << std::endl;
std::cout << (int) 16777217.0 << std::endl; 
std::cout << (int) std::pow(2,24) << std::endl;

16777217
16777216
16777217
16777216


  * using double precision is one solution (1 sign bit, 11-bit exponent, 52-bit significand), but costly for calculations done on vector units and GPUs.
  * Complex numerical calculations require care, especially when the developer does not know (or have control over) input parameters. Examples that you need to consider
    * Overflows/underflows
    * Precision loss due to accidental cancelation of terms

<a id='stl'></a>
# Using the Standard Template Library to reduce code complexity

  * The STL is a set of powerful, predefined abstract datatypes, functions, and algorithms designed to handle user-specified datatypes (as well as basic types).
  * By using them you can reduce the ammount of code you have to write and make it easier to optimize and maintain your code in the future (as the compiler will do that for you as it improves)
  

In [20]:
// vector is a resizeable container of any type
// For example, a vector of integers
std::vector<int> a(10,1.0);
print(&a[0],a.size());

1 1 1 1 1 1 1 1 1 1 


In [21]:
a.resize(20,2.0);
print(&a[0],a.size());

1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 


In [22]:

// A common pattern to avoid when possible
std::vector<int> a;
for (int i=0; i<10; i++)
    a.push_back(i*i);
print(&a[0],a.size());


0 1 4 9 16 25 36 49 64 81 


  * What is wrong with this syntax?

  * Memory is allocated when the vector length is increased. (in this simple example, the compiler likely optimizes it away)
  * Repeated allocations are slow and fragment memory
  * Use reserve to pre-allocate the size of the vector when you know the size of the vector

In [23]:

std::vector<int> a;
a.reserve(10);
for (int i=0; i<10; i++)
    a.push_back(i*i);
print(&a[0],a.size());


0 1 4 9 16 25 36 49 64 81 


# How should you loop over vectors? As with arrays, we can use recent language improvements to increase readability and performance

In [41]:
std::vector<int> a(20,3.0);

int sum=5;
for ( std::vector<int>::size_type i=0; i< a.size(); i++)
    sum += a[i];
std::cout << sum << std::endl;

65


In [26]:
int sum=5;
for ( auto i = a.begin(); i!=a.end(); i++)
    sum += *i;
std::cout << sum << std::endl;

65


In [27]:
int sum=5;
for ( int i: a)
    sum += a[i];
std::cout << sum << std::endl;

65


In [28]:
int sum=5;
for ( auto i: a)
    sum += a[i];
std::cout << sum << std::endl;

65


In [29]:
int sum=5;
for ( const auto &i: a)
    sum += a[i];
std::cout << sum << std::endl;

65


  * What is the benefit of adding ```const &```?

# The STL also provides many helpful functions. 

In [30]:
// In this case, std::accumulate
// lets you avoid writing the loop completely
// Note: Can be used for variables of other types. Anything with a "begin()" and "end()" function
int sum = std::accumulate(a.begin(), a.end(), 5);
std::cout << sum << std::endl;

65


In [31]:
//It is straightforward to customize which elements are considered. 
//For example, we can skip the first and last one
int sum = std::accumulate(a.begin()+1, a.end()-1, 5);
std::cout << sum << std::endl;
//but beware of overflows and underflows if the length of the vector is unknown...

59


# You can also do more customized calculations with minimal code

In [42]:
int sq(int c, int d) {return c + d*d;} 
int sum = std::accumulate(a.begin(), a.end(), 0,sq);
std::cout << sum << std::endl;

180


In [43]:
auto lambda = [&](int c, int d){return c + d*d; };
int sum = std::accumulate(a.begin(), a.end(), 0,lambda);
std::cout << sum << std::endl;

180


  * Prefer lambda syntax here... More on lambdas later

  * Lets investigate other functions provided by the algorithm package https://en.cppreference.com/w/cpp/algorithm

# There is lots more beyond vectors in the STL

  * array (prefer array to vector when possible)
  * string (prefer string to char*, TString, or other custom string classes)
  * map
  * unordered_map (prefer unordered_map to map when possible)

# Arrays are vectors that are not resizeable

In [44]:
//Initialization works just like before
std::array<int, 10> a{1,2,3,4,5,6,7,8,9,10};
print(&a[0],a.size());


1 2 3 4 5 6 7 8 9 10 


# Why use arrays?

  * Compared to using ```std::vector```, ```std::array``` will be more memory efficient when you know the size of your array. std::array has zero memory overhead compared to raw arrays
  * Compared to using raw arrays: like vector, ```std::array``` provides bounds checking and helper functions for things like copying and looping
  * HEP note: Last I tried, ROOT serialization has a love-hate relationship with std::array. Use with care for use cases that 

# An example of added safety using std::array

In [45]:
int sum(const std::array<int64_t, 10> &a) {
   return std::accumulate(a.begin(),a.end(),0);
}

std::array<int64_t, 10> a{1,2,3,4,5,6,7,8,9,10};
std::cout << sum(a) << std::endl;

55


  * No need to worry about getting the wrong array length. The function interface enforces that the array should be length 10 (and provides a self documenting interface)

# Strings

In [46]:
// Creating a string from const char*
std::string str1 = "hello";
 
// Creating a string using string literal
auto str2 = "world"s;
 
// Concatenating strings
std::string str3 = str1 + " " + str2;
 
// Print out the result
std::cout << str3 << '\n';

hello world


In [37]:
 
std::string::size_type pos = str3.find(" ");
str1 = str3.substr(pos + 1); // the part after the space
str2 = str3.substr(0, pos);  // the part till the space
 
std::cout << str1 << ' ' << str2 << '\n';
 

world hello


In [38]:
// Accessing an element using subscript operator[]
std::cout << str1[0] << '\n';
str1[0] = 'W';
std::cout << str1 << '\n';

w
World


  * As we saw with std::vector, there are more and more helper functions that facilitate string manipulations in newer C++ standards. (but use Python if thats the primary purpose of your program...)  

# map and unordered_map

  * These containers do what their name suggests - store key-value pairs.
  * ```std::map``` performance is O(log(n)) and elements are ordered
  * ```std::unordered_map``` uses a hash table with performance between O(1) and O(n)
  * Prefer std::vector to either when possible in performance sensitive applications (different from Python).

In [39]:
std::unordered_map<std::string, std::string> u = {
     {"RED","#FF0000"},
     {"GREEN","#00FF00"},
     {"BLUE","#0000FF"}
};
 
// Helper lambda function to print key-value pairs
auto print_key_value = [](const auto& key, const auto& value) {
    std::cout << "Key:[" << key << "] Value:[" << value << "]\n";
};
 
std::cout << "\nIterate and print key-value pairs using C++17 structured binding:\n";
for( const auto& [key, value] : u )
    print_key_value(key, value);


Iterate and print key-value pairs using C++17 structured binding:
Key:[BLUE] Value:[#0000FF]
Key:[GREEN] Value:[#00FF00]
Key:[RED] Value:[#FF0000]


In [40]:

// Add two new entries to the unordered_map
u["BLACK"] = "#000000";
u["WHITE"] = "#FFFFFF";
 
std::cout << "\nCheck what we just did:\n"
             "The HEX of color RED is: " << u["RED"] << "\n"
             "The HEX of color BLACK is: " << u["BLACK"] << "\n";


Check what we just did:
The HEX of color RED is: #FF0000
The HEX of color BLACK is: #000000


In [47]:
std::cout << "\nIterate and print key-value pairs using C++17 structured binding:\n";
for( const auto& [key, value] : u )
    print_key_value(key, value);
 


Iterate and print key-value pairs using C++17 structured binding:
Key:[WHITE] Value:[#FFFFFF]
Key:[BLACK] Value:[#000000]
Key:[BLUE] Value:[#0000FF]
Key:[GREEN] Value:[#00FF00]
Key:[RED] Value:[#FF0000]


<a id='memory'></a>
# C++ and memory management

* C++ requires programmers to think about memory management. HEP codes are often running many distinct algorithms together in an application. In this case, memory is particularly important to optimize.

<center><img src="./images/stack_heap.jpeg" width="400" /></center>

# How to distinguish stack and heap?

|                      | Stack                          | Heap             |
|----------------------|--------------------------------|------------------|
| Memory<br>Allocation | Contiguous blocks <br> Last in first out allocation           | Fragmented. Memory <br> is allocated in random order       |
| Access speed         | Faster  (O(1) time)                       | Slower (O(N) time)          |
| Who allocates?       | Compiler                       | Programmer       |
| Access privledges    | Local variables                | Global variables |
| Deallocation         | Automatic <br>(variable scope) | Manual           |
| Threads              | Each thread has own stack     | Shared across threads |
| Gotcha               | Limited memory                 | Fragementation   |

  * Prefer using the stack. Use heap memory when you need to allocate a large amount of memory or when you need memory that will persist beyond the lifetime of the variable that allocated it. 

# Lets come back to std::vector and using push_back

  * This is how memory gets allocated as vectors grow via push_back (or emplace_back)

<center><img src="./images/stdvector.png" width="400" /></center>

  * Each time the vector grows past its current allocation, its allocation is doubled and all values are copied from the old memory allocation to the new one.
    * Therefore it is faster to allocate a single large allocation instead of multiple/many small

<a id='pointers'></a>
# Pointers

  * A "pointer" (eg, ```int*```) is a value referring to a memory location
  * A pointer is allocated on the stack and points to memory located on the heap
  * "Dereference" a pointer to get the value stored in that memory location
  * Pointers support operators (```+```,```-```,```++```,```--```), subscript operators ```[]``` and comparison operators (```==```, ```!=```)

In [48]:
int64_t* ptr = nullptr; // avoid ptr=0 for type safety checks

ptr = new int64_t; // create an integer on the heap
*ptr = 10;
std::cout << *ptr << std::endl;
delete ptr; //always clean up (unless some other variable takes ownership)

10


In [62]:
// Similarly you can allocate an array
int* myArray = new int[10]; //create an array of integers on the heap
myArray[2]=10;
int myInt = myArray[2];
std::cout << myInt << std::endl;
delete [] myArray; //always clean up (unless some other variable takes ownership)

10


<center><img src="./images/pointers_stack_heap.png" width="400" /></center>

In [63]:
int vals[4] = {1, 2, 3, 4}; 
int *ptr = vals;
std::cout << ptr[1] << std::endl; 

2


In [64]:
// Two other ways to say the same thing

std::cout << *(ptr + 1) << std::endl; 

int* ptr1 = ptr + 2; 
std::cout << ptr1[-1] << std::endl;

2
2


In [65]:
// We can also inspect the memory location
std::cout << ptr << std::endl;

0x7f7369976000


In [52]:
// operators advance in memory by sizeof(int> (or whatever type is being pointed to)
std::cout << ptr + 1 << std::endl; 

0x7f736999b004


# Best practice - use modern pointers when possible

  * ```std::unique_ptr``` and ```std::shared_ptr``` let you avoid worrying about memory management. As it is easy to forget to delete memory you allocated, this is very convienent.
      


# std::unique_ptr

In [53]:
#include <memory>
std::unique_ptr<int> p{ new int{10} };

//Or avoid "new" completely
std::unique_ptr<int> p1 = std::make_unique<int>(11);

//Print the value and get the memory location
std::cout << *p << " " << p.get() << " " << *p1 << " " << p1.get() << std::endl;

10 0x7f731fa13d20 11 0x7f731ffed410


# unique_ptrs are *unique*

In [54]:
std::unique_ptr<int> p2;
p2=p;

input_line_104:3:3: error: overload resolution selected deleted operator '='
p2=p;
~~^~
/home/conda/feedstock_root/build_artifacts/root_base_1679383268638/_build_env/x86_64-conda-linux-gnu/include/c++/11.3.0/bits/unique_ptr.h:469:19: note: candidate function has been explicitly deleted
      unique_ptr& operator=(const unique_ptr&) = delete;
                  ^
/home/conda/feedstock_root/build_artifacts/root_base_1679383268638/_build_env/x86_64-conda-linux-gnu/include/c++/11.3.0/bits/unique_ptr.h:371:19: note: candidate function not viable: expects an rvalue for 1st argument
      unique_ptr& operator=(unique_ptr&&) = default;
                  ^
/home/conda/feedstock_root/build_artifacts/root_base_1679383268638/_build_env/x86_64-conda-linux-gnu/include/c++/11.3.0/bits/unique_ptr.h:386:2: note: candidate function [with _Up = int, _Ep = std::default_delete<int>] not viable: expects an rvalue for 1st argument
        operator=(unique_ptr<_Up, _Ep>&& __u) noexcept
        ^
/home/conda/fe

# Instead their ownership can be transfered

In [55]:
std::unique_ptr<int> p2(std::move(p));
std::cout << *p2 << " " << p2.get() << " " << std::endl;

10 0x7f731fa13d20 


  * When the ```unique_ptr``` (```p2```) goes out of scope, the memory it points to is cleaned up
  

# shared_ptrs are a reference counted alternative

  * Memory is cleaned up when no variable refers to the memory

In [56]:
auto sptr = std::make_shared<int>(12);
auto sptr2 = sptr; //now this works and creates another "reference"
std::cout << sptr.use_count() << " " << (*sptr) << " " << ++(*sptr) << std::endl;

2 12 13


# Using lambda functions
<a id='lambdas'></a>

  * Lambdas are inline local-scope function objects. Use them to increase code readability and code "locality" 
  * C++11 introduced lambda functions into the standard. Their abilities have increased substantially with each standard revision

Today I have just a short introduction to lambdas
  
```  auto x = [capture clause] (parameters) { body }```

  * The ```capture clause``` marks the declaration of the lambda and how the local scope arguments are captured (by-value, by-reference, etc.) - can be empty
  * ```Parameters``` are normal function parameters (can be empty) 
  * The ```body``` is a normal function body
  * The object created by the lambda expression is ```x```

In [57]:
#include <algorithm>
#include <array>
std::array<int,5> myarray{5, 4, 3, 2, 1};

// a named lambda 
auto lambda = [](int a, int b){ return a < b; }; 

std::sort(myarray.begin(), myarray.end(), lambda);
print(&myarray[0],myarray.size());

1 2 3 4 5 


In [58]:
// in alternative, in one line of code: 
// aka, an unnamed lambda 
std::sort(myarray.begin(), myarray.end(), [](int a, int b){ return a > b; });
print(&myarray[0],myarray.size());

5 4 3 2 1 


# What do I mean "capture"?

Lambdas can also use (aka, "capture") one or more external variables
  * ```[]``` means capture nothing
  * ```[=]``` captures all variables by-value
  * ```[&]``` captures all variables by-reference
  * ```[v1]``` captures only ```v1``` by-value
  * ```[&v1]``` captures only ```v1``` by-reference
  * ```[v1, &v2]``` captures ```v1``` by-value and ```v2``` by-reference
  * Combinations like ```[=, &v2]``` (capture all but v2 by value, v2 by ref) and ```[&, v2]``` (capture all but v2 by ref, v2 by vaue) are also valid 

# More complex examples

In [59]:
float minVal=10.;
std::array<float,5> vals={1.0, 11.0, 13.0, 5.0, -14.0};

//Does my array have at least one element > minVal?
auto lambda1 = [&minVal](int value) { return value > minVal; };   
auto output= std::find_if(vals.begin(), vals.end(), lambda1);

if ( output != vals.end() ) {
    std::cout << *output << std::endl;
}

11


# From C++17, lambdas support constexpr

  * ```constexpr``` tells the compiler to do the calculation at compile time
  * A good use case is factorial functions. I once debugged performance issues on a code that spent 5% of its time computing factorials. There are not very many factorial calculations those answer fits into an int..

In [60]:
// constexpr lambda
constexpr auto factorial = [](int value) constexpr { 
    int ret = 1;
    for (int i = 2; i <= value; i++) ret *= i;
    return ret; 
};
constexpr int v1 = factorial(4); // '24'
std::cout << v1 << std::endl;

24


In [61]:
constexpr int f() {
    return factorial(3); // 6
}

std::cout << f() << std::endl;

6


# Thats the end of this section. Any more questions?